In [15]:
import logging
import neo4j


def get_neo4j_connection() -> neo4j.Session:
    """
    Gets a session to neo4j
    """
    driver = neo4j.GraphDatabase.driver(
        "bolt://localhost:7687/", auth=("neo4j", "neo4j222")
    )
    try:
        driver.verify_connectivity()
        session = driver.session(database="neo4j")
        return session
    except Exception as connection_error:
        logging.error("Failed to establish session to neo4j", connection_error)
        session.close()
        driver.close()

In [10]:
def load_intersection_relations() -> None:
    """
    Load the roads into neo4j

    :param db_session: Session for a neo4j connection
    :return: None
    """
    logging.info("Loading road network connections into neo4j")
    db_session = get_neo4j_connection()
    load_intersection_relation_query = """
    LOAD CSV WITH HEADERS FROM 'file:///mexico_relations.csv' AS segment
    CALL {
        WITH segment
        MATCH(intersection_a:Intersection {osmid: segment.u})
        MATCH(intersection_b: Intersection {osmid: segment.v})
        MERGE (intersection_a)-[road_segment:ROAD_SEGMENT {osmid: segment.osmid}]->(intersection_b)
        SET road_segment.oneway = segment.oneway,
        road_segment.ref = segment.ref,
        road_segment.name = segment.name,
        road_segment.highway = segment.highway,
        road_segment.max_speed = segment.maxspeed,
        road_segment.length = toFloat(segment.length)
    } IN TRANSACTIONS OF 1000 ROWS;
    """
    db_session.run(load_intersection_relation_query)
    
load_intersection_relations()

TransientError: {code: Neo.TransientError.General.MemoryPoolOutOfMemoryError} {message: Unable to complete transaction.: The memory pool limit was exceeded. The corresponding setting can be found in the error message}

In [17]:
def count_load_addresses() -> None:
    logging.info("Loading addresses into Neo4j and count")
    
    db_session = get_neo4j_connection()
    
    load_address_query_count = '''
    
    CALL apoc.load.json("https://cdn.neo4jlabs.com/data/addresses/san_mateo.geojson") YIELD value
    RETURN COUNT(value) AS num
    '''
    db_session.run(load_address_query_count)

In [18]:
count_load_addresses()

In [11]:
def create_indices() -> None:
    """
    Creates indices for faster lookup

    :return: None
    """
    logging.info("Creating indices for addresses")

    db_session = get_neo4j_connection()
    address_constraint_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Address) REQUIRE a.id IS UNIQUE"
    db_session.run(address_constraint_query)

def load_addresses() -> None:
    """
    Loads addresses from OpenAddresses and attaches them to the
    nearest intersection

    :return: None
    """
    logging.info("Loading addresses into neo4j...")

    db_session = get_neo4j_connection()
    load_address_query = '''
    CALL apoc.periodic.iterate(
        'CALL apoc.load.json("file:///source.geojson") YIELD value',
        'CREATE (a:Address {id: value.properties.id})
        SET a.location = 
            point({latitude: value.geometry.coordinates[1], longitude: value.geometry.coordinates[0]}),
            a.full_address = value.properties.number + " " + value.properties.street + " " + value.properties.city + ", MX " + value.properties.postcode
        SET a += value.properties',
    {batchSize:10000, parallel:true})
    '''
    db_session.run(load_address_query)

In [12]:
create_indices()

In [16]:
load_addresses()